# RabbitMQ in python - pika

### Jak uruchomić programy?
Uruchom serwer RabbitMQ (np. przez `rabbitmq-server` w terminalu, jeśli jest zainstalowany).

#### Uruchom konsumentów:
* Dla punkt-punkt: `python point_to_point_consumer.py.`

* Dla jeden-do-wielu: uruchom kilka instancji `python one_to_many_consumer.py` w osobnych terminalach.

#### Uruchom producentów:
* `python point_to_point_producer.py.`

* `python one_to_many_producer.py.`

#### Dlaczego te rozwiązania są poprawne?
* Punkt-punkt: 
* * Używa jednej kolejki do bezpośredniej komunikacji.

* * Mechanizm potwierdzenia zapewnia, że wiadomości są przetwarzane niezawodnie.

* Jeden-do-wielu: 
* * Wymiana fanout rozsyła wiadomości do wszystkich konsumentów.

* * Każdy konsument ma własną kolejkę, co pozwala na równoczesny odbiór.

Te rozwiązania są proste, zgodne z celem zadania i pokazują podstawowe mechanizmy RabbitMQ w sposób 

In [ ]:
import pika

# Nawiązanie połączenia z serwerem RabbitMQ
connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()

# Deklaracja kolejki
channel.queue_declare(queue='point_to_point_queue')

# Wysłanie wiadomości
for i in range(5):
    message = f"Wiadomość {i}"
    channel.basic_publish(exchange='', routing_key='point_to_point_queue', body=message.encode())
    print(f"Wysłano: {message}")

# Zamknięcie połączenia
connection.close()

In [ ]:
import pika

# Funkcja obsługująca odebrane wiadomości
def callback(ch, method, properties, body):
    print(f"Odebrano: {body.decode()}")
    # Potwierdzenie przetworzenia wiadomości
    ch.basic_ack(delivery_tag=method.delivery_tag)

# Nawiązanie połączenia z serwerem RabbitMQ
connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()

# Deklaracja kolejki
channel.queue_declare(queue='point_to_point_queue')

# Ustawienie konsumenta
channel.basic_consume(queue='point_to_point_queue', on_message_callback=callback)

print('Oczekiwanie na wiadomości. Aby zakończyć, naciśnij CTRL+C')
channel.start_consuming()

In [ ]:
import pika

# Nawiązanie połączenia z serwerem RabbitMQ
connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()

# Deklaracja wymiany typu fanout
channel.exchange_declare(exchange='fanout_exchange', exchange_type='fanout')

# Wysłanie wiadomości
for i in range(5):
    message = f"Wiadomość rozgłoszeniowa {i}"
    channel.basic_publish(exchange='fanout_exchange', routing_key='', body=message.encode())
    print(f"Wysłano: {message}")

# Zamknięcie połączenia
connection.close()

In [ ]:
import pika

# Funkcja obsługująca odebrane wiadomości
def callback(ch, method, properties, body):
    print(f"Odebrano: {body.decode()}")

# Nawiązanie połączenia z serwerem RabbitMQ
connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()

# Deklaracja wymiany typu fanout
channel.exchange_declare(exchange='fanout_exchange', exchange_type='fanout')

# Deklaracja tymczasowej kolejki
result = channel.queue_declare(queue='', exclusive=True)
queue_name = result.method.queue

# Powiązanie kolejki z wymianą
channel.queue_bind(exchange='fanout_exchange', queue=queue_name)

# Ustawienie konsumenta
channel.basic_consume(queue=queue_name, on_message_callback=callback, auto_ack=True)

print('Oczekiwanie na wiadomości. Aby zakończyć, naciśnij CTRL+C')
channel.start_consuming()